In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm
import matplotlib.ticker as ticker
import seaborn as sns

from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

import os

from retry import retry

import string

from collections import Counter

import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import pos_tag
from nltk import download
download('punkt')
download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import Text as nltk_text

In [ ]:
about_url = 'https://bigbangtrans.wordpress.com/about/'

page = rq.get(about_url)
soup = BeautifulSoup(page.text, features="html.parser")

links = []
for link in soup.find_all('a'):
    if link.parent.name == 'li':
        links.append(link.get('href'))

episodes = []
for link in links:
    if 'episode' in link:
        episodes.append(link)
episodes

In [ ]:
@retry()
def parser(num):
    episode = rq.get(episodes[num])
    soup = BeautifulSoup(episode.text, features="html.parser")
    episode_name = soup.find('h2').text
    season_number = episode_name[7:9]
    episode_number = episode_name[18:20]
    text = []
    for k in soup.find_all('p'):
        text.append(k.text)
    episode_text = ' '.join(text)
    episode_text = episode_text.replace('\xa0', ' ')
    return season_number, episode_number, episode_text

In [ ]:
scripts = []
for num in range(len(episodes)):
    one_script = parser(num)
    scripts.append(one_script)

df = pd.DataFrame(scripts)
df.head()
df.drop_duplicates(inplace=True)
df.columns = ['season', 'episode', 'script']
df

In [ ]:
df.to_csv('TBBT.csv', sep = ',', encoding='utf-8')
df.to_excel('TBBT excel.xlsx')

#запускать отсюда

In [ ]:
#стоп-слова - имена персонажей
characters_full_names = [
    'Leonard Hofstadter',
    'Sheldon Cooper',
    'Penny',
    'Howard Wolowitz',
    'Raj Koothrappali',
    'Bernadette Rostenkowski Wolowitz',
    'Amy Farrah Fowler',
    'Stuart Bloom'
    ]

clean_names = []
for name in characters_full_names:
    name = name.lower().split()
    name_clean = [word for word in name]
    clean_names.append(name_clean)

characters_stop_names = []
for full_name in clean_names:
    for item in full_name:
        characters_stop_names.append(item)
characters_stop_names.remove('bloom')

#стоп-слова - частотные слова из сериала, которые я не посчитала информативными
my_stop_words = []
with open('my stopwords.txt', 'r', encoding='utf-8') as f:
  for i in f:
    my_stop_words.append(i.strip())

#стоп-слова - 50 самых распространенных глаголов в английском
#источник: https://www.ef.com/wwen/english-resources/english-vocabulary/top-50-verbs/
#я немного изменила этот список
common_verbs = []
with open('50 most common verbs in English.txt', 'r', encoding='utf-8') as f:
  for i in f:
    common_verbs.append(i.strip())

#собираем все стоп-слова
stop_words = stopwords.words('english') + characters_stop_names + my_stop_words + common_verbs

In [ ]:
def clean(text):
    text = text.replace('Create a free website or blog at WordPress.com', '')
    text = text.replace('Blog at WordPress.com', '')
    text = text.lower()
    text_clean = word_tokenize(text)
    text_clean = [word for word in text_clean if word[0].isalpha()]
    text_clean = [word for word in text_clean if word not in stop_words]
    return text_clean

In [ ]:
data = pd.read_csv('TBBT.csv')
clean_script = []
script = data['script'].to_list()

for episode in script:
    clean_episode = clean(episode)
    clean_script.append(clean_episode)

data['clean script'] = clean_script

In [ ]:
#пример для того, чтобы убедиться, что все получилось
res = data['clean script'].to_list()
print(res[10])

In [ ]:
#не запускать
# nltk без частеречной разметки

lemmatized = []
not_lemmatized = data['clean script'].to_list()

for ep in not_lemmatized:
    lemmatized_episode = []
    for word in ep:
        lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_episode.append(lemmatized_word)
    lemmatized.append(lemmatized_episode)
lemmatized_result = [word for word in lemmatized if word not in stop_words]
#data['nltk lemmatized script'] = lemmatized_result

#lemmatized_result[10]

In [ ]:
#nltk с частеречной разметкой - объявление функций
def pos_tagger(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_text(text):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(word_tokenize(text)):
        wordnet_pos = pos_tagger(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))
    return res

In [ ]:
#nltk с частеречной разметкой - лемматизация
#каждое слово лемматизируется отдельно и добавляется в список-эпизод
lemmatized_tagged = []
not_lemmatized = data['clean script'].to_list()

for ep in not_lemmatized:
    lemmatized_episode_tagged = []
    for word in ep:
        lemmatized_episode_tagged += lemmatize_text(word)
    lemmatized_tagged.append(lemmatized_episode_tagged)

#lemmatized_tagged[10]

In [ ]:
lemmatized_tagged_result = []
for l in lemmatized_tagged:
    res = []
    for word in l:
        if word not in stop_words:
            res.append(word)
    lemmatized_tagged_result.append(res)

data['nltk lemmatized script (with pos tagging)'] = lemmatized_tagged_result

In [ ]:
#не запускать
#с этим кодом не удаляются стоп-слова
lemmatized_tagged_result = [word for word in lemmatized_tagged if word not in stop_words]
data['nltk lemmatized script (with pos tagging)'] = lemmatized_tagged_result

In [ ]:
data

In [ ]:
#пример для того, чтобы убедиться, что все получилось
res = data['nltk lemmatized script (with pos tagging)'].to_list()
print(res[207])

In [ ]:
#частотные слова во всем сериале
nltk_lemmatized_script = data['nltk lemmatized script (with pos tagging)'].to_list()
nltk_full_script = []
for ep in nltk_lemmatized_script:
    for list in ep:
        nltk_full_script.append(list)
counter_series = Counter(nltk_full_script)
counter_series_100 = Counter(nltk_full_script).most_common(100)

#print(counter_series_100)

In [ ]:
#частотные слова в каждом эпизоде
counter_episode = []
counter_episode_10 = []
for ep in nltk_lemmatized_script:
    counter_episode.append(Counter(ep))
    counter_episode_10.append(Counter(ep).most_common(10))

data['counter'] = counter_episode
data['10 most common words'] = counter_episode_10

#print(counter_episode_10[:10])

In [ ]:
#биграммы во всем сериале
bigrams_series = Counter(nltk.bigrams(nltk_full_script))
bigrams_series_50 = bigrams_series.most_common(50)

#bigrams_series
#bigrams_series_50

In [ ]:
#биграммы в каждой серии
bigrams_episode = []
bigrams_episode_10 = []
for ep in nltk_lemmatized_script:
    bigrams_episode.append(Counter(nltk.bigrams(ep)))
    bigrams_episode_10.append(Counter(nltk.bigrams(ep)).most_common(10))
data['all bigrams'] = bigrams_episode
data['5 most common bigrams'] = bigrams_episode_10

In [ ]:
#выявление частотных слов и биграмм отдельно в сезонах 1-3 и 4-10
seasons_1_3 = data[:63]
seasons_4_10 = data[63:]

script_1_3 = []
for ep in nltk_lemmatized_script[:63]:
    for list in ep:
        script_1_3.append(list)

script_4_10 = []
for ep in nltk_lemmatized_script[63:]:
    for list in ep:
        script_4_10.append(list)

counter_1_3 = Counter(script_1_3)
counter_1_3_100 = counter_1_3.most_common(100)

counter_4_10 = Counter(script_4_10)
counter_4_10_100 = counter_4_10.most_common(100)

bigrams_1_3 = Counter(nltk.bigrams(script_1_3))
bigrams_1_3_50 = bigrams_1_3.most_common(50)

bigrams_4_10 = Counter(nltk.bigrams(script_4_10))
bigrams_4_10_50 = bigrams_4_10.most_common(50)

In [ ]:
#проверяем, есть ли слова в списке 100 частотных слов сезонов 1-3
def check(text):
    flag = False
    for word in counter_1_3_100:
        for i in word:
            if i == text:
                flag = True
    return flag

check(input())

In [ ]:
#начисление очков за совпадение частотных слов и биграмм сезонов 4-10 со словами и биграммами сезонов 1-3
#считается для всех серий для упрощения упорядочивания данных
counter_4_10_points = []
for ep in counter_episode_10:
    score = 0
    for word in ep:
      for pair in counter_1_3_100:
        if word[0] == pair[0]:
            score += 1
    counter_4_10_points.append(score)


bigrams_4_10_points = []
for ep in bigrams_episode_10:
    score = 0
    for bigram in ep:
      for pair in bigrams_1_3_50:
        if bigram[0] == pair[0]:
            score += 1
    bigrams_4_10_points.append(score)

data['points for words from seasons 1-3'] = counter_4_10_points
data['points for bigrams from seasons 1-3'] = bigrams_4_10_points

#counter_4_10_points[63:]
#bigrams_4_10_points[63:]

In [ ]:
#среднее значение
print(np.average(counter_4_10_points[63:]))
print(np.average(bigrams_4_10_points[63:]))

In [ ]:
#график с частотными биграммами всех сезонов
plt.plot(data['points for bigrams from seasons 1-3'])

#plt.savefig('points_for_common_bigrams.png', bbox_inches='tight')

#plt.show()

In [ ]:
#график с частотными биграммами только сезонов 4-10
plt.plot(data['points for bigrams from seasons 1-3'][63:])

#plt.show()

In [ ]:
#график с частотными словами всех сезонов
plt.plot(data['points for words from seasons 1-3'])

#plt.savefig('points_for_common_words.png', bbox_inches='tight')

#plt.show()

In [ ]:
#график с частотными словами только сезонов 4-10
plt.plot(data['points for words from seasons 1-3'][63:])

#plt.show()

**практическое применение**

In [ ]:
#в результате серии, где встречается хотя бы одно слово из запроса
search = input('Введите слова без запятых:').split()
season = data['season'].to_list()
episode = data['episode'].to_list()
for ep in range(len(nltk_lemmatized_script)):
    for word in search:
        if word in nltk_lemmatized_script[ep]:
            print(season[ep], episode[ep], word)

In [ ]:
#в результате только серии, где встречаются все слова из запроса
search = input('Введите слова без запятых:').split()
season = data['season'].to_list()
episode = data['episode'].to_list()
for ep in range(len(nltk_lemmatized_script)):
    counter = 0
    for word in search:
        if word in nltk_lemmatized_script[ep]:
            counter += 1
    if counter == len(search):
        print(season[ep], episode[ep])

In [ ]:
#облако слов
data['counter'] = counter_episode
def get_wordcloud(num):
    wordcloud = WordCloud(width = 1500,
                      height = 1000,
                      include_numbers = False,
                      background_color='black',
                      colormap='magma').generate(', '.join(counter_episode[num]))
    plt.figure(figsize=(30, 20)) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.show()

In [ ]:
get_wordcloud(int(input()))

In [ ]:
data

In [ ]:
data.to_csv('TBBT with data.csv', sep = ',', encoding='utf-8')

**идеи на будущее**

соотнесение оценок пользователей и ключевых слов серии

In [ ]:
#начисление очков за совпадение частотных слов серии и сериала
#для сравнения с оценками пользователей
counter_points = []
for ep in counter_episode_10:
    score = 0
    for word in ep:
      for pair in counter_series_100:
        if word[0] == pair[0]:
            score += 1
    counter_points.append(score)
counter_points

In [ ]:
#начисление очков за совпадение частотных биграм серии и сериала
#для сравнения с оценками пользователей
bigrams_points = []
for ep in bigrams_episode_10:
    score = 0
    for bigram in ep:
      for pair in bigrams_series_50:
        if bigram[0] == pair[0]:
            score += 1
    bigrams_points.append(score)
bigrams_points

тематическое моделирование

In [ ]:
import gensim
from collections import defaultdict
from gensim.corpora import Dictionary
from gensim import models
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from gensim import corpora

In [ ]:
texts = data['nltk lemmatized script (with pos tagging)'].to_list()
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

tfidf = models.TfidfModel(corpus)

corpus_tfidf = tfidf[corpus]

lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)
corpus_lsi = lsi_model[corpus_tfidf]

lsi_model.print_topics(5)